In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
sc=SparkContext(appName="hdfs_dataframe")          #初始化SparkContext 
spark=SparkSession.builder.appName('hdfs_dataframe').master("Yct201811021847").getOrCreate()

In [2]:
movies_df=spark.read.csv('data/movies.csv',header=True, inferSchema=True).dropna()

In [3]:
ratings_df=spark.read.csv('data/ratings.csv',header=True,inferSchema=True).dropna()

In [4]:
ratings_df.printSchema()
movies_df.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



In [5]:
movies_df.createOrReplaceTempView('movies')      # 创建movies视图
ratings_df.createOrReplaceTempView('ratings')       # 创建ratings视图

In [6]:
spark.sql("SELECT * FROM ratings")
spark.sql("SELECT * FROM movies")
rank_data=spark.sql(" SELECT title , rating , userID, genres ,avg_movie_rating , \
        DENSE_RANK() OVER( ORDER BY avg_movie_rating desc) rank FROM \
    (SELECT m.title , rating , userID, m.genres, \
    AVG(rating) OVER(partition by m.movieId) avg_movie_rating \
    FROM  movies m LEFT JOIN ratings r ON r.movieId = m.movieId \
    WHERE rating is not null ) A ")#.filter('avg_movie_rating is not null')

In [7]:
rank_data.filter('rank =2 ').show()

+--------------------+------+------+------+----------------+----+
|               title|rating|userID|genres|avg_movie_rating|rank|
+--------------------+------+------+------+----------------+----+
|Trial, The (Procè...|   5.0|    59| Drama|             4.9|   2|
|Trial, The (Procè...|   5.0|   156| Drama|             4.9|   2|
|Trial, The (Procè...|   4.5|   330| Drama|             4.9|   2|
|Trial, The (Procè...|   5.0|   410| Drama|             4.9|   2|
|Trial, The (Procè...|   5.0|   469| Drama|             4.9|   2|
+--------------------+------+------+------+----------------+----+



In [8]:
top5_rank=rank_data.filter('rank < 5')   #提取排名前10的

In [9]:
top5_rank.show(10)

+--------------------+------+------+--------------------+----------------+----+
|               title|rating|userID|              genres|avg_movie_rating|rank|
+--------------------+------+------+--------------------+----------------+----+
|Awfully Big Adven...|   5.0|   191|               Drama|             5.0|   1|
|What Happened Was...|   5.0|   191|Comedy|Drama|Roma...|             5.0|   1|
|Strictly Sexual (...|   5.0|    68|Comedy|Drama|Romance|             5.0|   1|
| The Love Bug (1997)|   5.0|    89|Adventure|Childre...|             5.0|   1|
|Raise Your Voice ...|   5.0|   175|             Romance|             5.0|   1|
|One I Love, The (...|   5.0|   598|Comedy|Drama|Romance|             5.0|   1|
|   The Editor (2015)|   5.0|    89|Comedy|Horror|Mys...|             5.0|   1|
|Tenchi Muyô! In L...|   5.0|    89|    Animation|Comedy|             5.0|   1|
|Martin Lawrence L...|   5.0|   553|  Comedy|Documentary|             5.0|   1|
|Who Killed Chea V...|   5.0|   462|    

In [10]:
pd_rank=top5_rank.toPandas()

In [11]:
pd_rank

,title,rating,userID,genres,avg_movie_rating,rank
0,"Awfully Big Adventure, An (1995)",5.0,191,Drama,5.00,1
1,What Happened Was... (1994),5.0,191,Comedy|Drama|Romance|Thriller,5.00,1
2,Strictly Sexual (2008),5.0,68,Comedy|Drama|Romance,5.00,1
3,The Love Bug (1997),5.0,89,Adventure|Children|Comedy|Fantasy,5.00,1
4,Raise Your Voice (2004),5.0,175,Romance,5.00,1
5,"One I Love, The (2014)",5.0,598,Comedy|Drama|Romance,5.00,1
6,The Editor (2015),5.0,89,Comedy|Horror|Mystery,5.00,1
7,Tenchi Muyô! In Love (1996),5.0,89,Animation|Comedy,5.00,1
8,Martin Lawrence Live: Runteldat (2002),5.0,553,Comedy|Documentary,5.00,1
9,Who Killed Chea Vichea? (2010),5.0,462,Documentary,5.00,1


In [12]:
pd_rank.drop_duplicates(subset=['title'], keep='first')

,title,rating,userID,genres,avg_movie_rating,rank
0,"Awfully Big Adventure, An (1995)",5.0,191,Drama,5.000000,1
1,What Happened Was... (1994),5.0,191,Comedy|Drama|Romance|Thriller,5.000000,1
2,Strictly Sexual (2008),5.0,68,Comedy|Drama|Romance,5.000000,1
3,The Love Bug (1997),5.0,89,Adventure|Children|Comedy|Fantasy,5.000000,1
4,Raise Your Voice (2004),5.0,175,Romance,5.000000,1
5,"One I Love, The (2014)",5.0,598,Comedy|Drama|Romance,5.000000,1
6,The Editor (2015),5.0,89,Comedy|Horror|Mystery,5.000000,1
7,Tenchi Muyô! In Love (1996),5.0,89,Animation|Comedy,5.000000,1
8,Martin Lawrence Live: Runteldat (2002),5.0,553,Comedy|Documentary,5.000000,1
9,Who Killed Chea Vichea? (2010),5.0,462,Documentary,5.000000,1


In [13]:
pd_rank['genres']

0                                      Drama
1              Comedy|Drama|Romance|Thriller
2                       Comedy|Drama|Romance
3          Adventure|Children|Comedy|Fantasy
4                                    Romance
5                       Comedy|Drama|Romance
6                      Comedy|Horror|Mystery
7                           Animation|Comedy
8                         Comedy|Documentary
9                                Documentary
10                                    Comedy
11                           Adventure|Drama
12                           Adventure|Drama
13                                     Drama
14                              Comedy|Drama
15                                    Comedy
16                               Documentary
17                             Crime|Mystery
18              Action|Comedy|Crime|Thriller
19                                     Drama
20                    Adventure|Comedy|Drama
21                                    Comedy
22        

In [15]:
import pandas as pd
pd.read_csv('category.csv')

,CategoryName,CategoryNum,OrderNum
0,Seafood,12,330
1,Meat/Poultry,6,173
2,Condiments,12,216
3,Confections,13,334
4,Produce,5,136
5,Dairy Products,10,366
6,Beverages,12,404
7,Grains/Cereals,7,196
